In [1]:
import numpy as np
import tensorflow as tf

tf.__version__

'1.14.0'

In [2]:
input_layer = tf.keras.layers.Input(shape=(28, 28, 3))
hidden_layer = tf.keras.layers.Dense(5)(input_layer)
output_layer = tf.keras.layers.BatchNormalization()(hidden_layer)

model = tf.keras.models.Model(input_layer, output_layer)

W1216 11:40:06.164047 25212 deprecation.py:506] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
model.get_updates_for(None)

[]

In [4]:
model.get_updates_for(model.inputs)

[<tf.Operation 'batch_normalization/AssignMovingAvg/AssignSubVariableOp' type=AssignSubVariableOp>,
 <tf.Operation 'batch_normalization/AssignMovingAvg_1/AssignSubVariableOp' type=AssignSubVariableOp>]

In [5]:
moving_mean, moving_variance = set(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)) - set(model.trainable_variables)

In [6]:
y_pred = model(model.input, training=True)
y_true = tf.placeholder(shape=(None, 28, 28, 5), dtype=tf.float32)

In [7]:
loss = tf.keras.losses.MeanSquaredError()(y_true, y_pred)
optimizer = tf.train.AdamOptimizer()

Keras APIs handle BatchNormalization updates to the moving_mean and moving_variance as part of their `fit()` and `evaluate()` loops. However, if a custom training loop is used with an instance of `Model`, these updates need to be explicitly included. Here's a simple example of how it can be done

```python
# model is an instance of Model that contains BatchNormalization layer.
update_ops = model.get_updates_for(None) + model.get_updates_for(model.inputs)
train_op = optimizer.minimize(loss)
train_op = tf.group([train_op, update_ops])
```

In [8]:
update_ops = model.get_updates_for(None) + model.get_updates_for(model.inputs)
train_op = optimizer.minimize(loss, var_list=model.trainable_variables)
train_op = tf.group([train_op, update_ops])

In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        sess.run(train_op, feed_dict={model.input: np.random.rand(32, 28, 28, 3), y_true: np.ones((32, 28, 28, 5))})
        print(sess.run(moving_mean))

[0.9905418  0.99049205 0.99014586 0.99012387 0.9907512 ]
[0.9811823  0.98107016 0.9803861  0.980346   0.98159504]
[0.9719129  0.97175294 0.97072697 0.9706639  0.97253674]
[0.96274096 0.96252525 0.961162   0.9610792  0.9635669 ]
[0.9536593  0.95338464 0.9516894  0.95158994 0.9546872 ]
[0.9446627  0.9443322  0.94231015 0.94219476 0.94589305]
[0.93575495 0.93536633 0.9330243  0.932892   0.9371808 ]
[0.9269324  0.92648613 0.9238289  0.9236804  0.92855275]
[0.9181988  0.9176942  0.91472435 0.91456157 0.9200096 ]
[0.9095512  0.90899396 0.90571284 0.9055313  0.9115434 ]
